# Introduction
In this notebook I will estimate minimal number of tokens that our llama model will consume per second.<br/>
The following calculations are made under optimistic assumption that we will parse only content from text fields.

In [10]:
import pandas as pd
from pathlib import Path
from transformers import LlamaTokenizerFast
from datetime import datetime

In [2]:
!python ../../tools/data_load.py coupons_1

^C
Traceback (most recent call last):
  File "/home/szymon/murmuras/ZPP_Murmuras/research/speed_reuirements_research/../../tools/data_load.py", line 101, in <module>
    children, parent, folder_names = create_fs_tree(service)
                                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/szymon/murmuras/ZPP_Murmuras/research/speed_reuirements_research/../../tools/data_load.py", line 44, in create_fs_tree
    .execute()
     ^^^^^^^^^
  File "/home/szymon/murmuras/ZPP_Murmuras/.venv/lib/python3.12/site-packages/googleapiclient/_helpers.py", line 130, in positional_wrapper
    return wrapped(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/szymon/murmuras/ZPP_Murmuras/.venv/lib/python3.12/site-packages/googleapiclient/http.py", line 923, in execute
    resp, content = _retry_request(
                    ^^^^^^^^^^^^^^^
  File "/home/szymon/murmuras/ZPP_Murmuras/.venv/lib/python3.12/site-packages/googleapiclient/http.py", line 191, in _retry_request
    resp

In [3]:
frames = []
DS_PATH = Path("../..") / "datasets" / "coupons_1"
PATHS = [
    DS_PATH / "lidl" / "Kopia test_data_2024_11_25_lidl_plus_content_generic_2024-12-05T07_39_49.726955559+01_00.csv",
    DS_PATH / "dm" / "Kopia test_data_2024_03_07_dm_content_generic_2024-12-05T10_09_32.502568365+01_00.csv",
    DS_PATH / "rewe" / "Kopia test_data_2024_03_07_rewe_content_generic_2024-12-05T10_30_59.948177782+01_00.csv",
    DS_PATH / "rossmann" / "Kopia test_data_2024_03_07_rossmann_content_generic_2024-12-05T10_24_07.981399375+01_00.csv"
]

for path in PATHS:
    frames.append(pd.read_csv(path))

In [18]:
tokenizer = LlamaTokenizerFast.from_pretrained("meta-llama/Llama-3.2-1B")
TEXT_COL_NAME = "text"
TIMESTAMP_COL_NAME = "seen_timestamp"

def count_tokens(text):
    return len(tokenizer.tokenize(text))

tokens_cum = 0
seconds_cum = 0

for frame in frames:

    texts = frame[TEXT_COL_NAME]
    texts = texts[texts.notnull()]
    total_tokens = texts.apply(count_tokens).sum()
    times = frame[TIMESTAMP_COL_NAME]
    times = times[times > 0]
    time_start = datetime.fromtimestamp(times.min() // 1000)
    time_end = datetime.fromtimestamp(times.max() // 1000)
    total_seconds = (time_end - time_start).total_seconds()

    print(total_tokens, total_seconds)
    tokens_cum += total_tokens
    seconds_cum += total_seconds

print(f"required min processing speed: {float(tokens_cum / seconds_cum)} tokens per second")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.


11739 169.0
15026 56.0
23708 80.0
14678 79.0
required min processing speed: 169.6640625 tokens per second
